In [15]:
import random
import numpy as np
import hashlib
import web3
import json

RQ 1 To what extend can pooled object hashes increase the transaction throughput and reduce cost for a fixity information storage service on the Ethereum blockchain?

RQ 2 What is the optimal pool size based on the corruption rates of digital objects in the archive in terms of transaction throughput and cost?

RQ 3 Given that metadata has a higher corruption rate, what effect has the split of metadata and objects on the operation cost?

In [16]:

# creation of N = 10k objects and assigning them corruption rate from 1% to 20% 
encoding = "utf-8"
N = 10
percent = 10
prevalence = (percent * N) /100.0
p = percent/100.0

class Object:
    def __init__(self,id,corruption_rate,pool_id=0):
        self.id=id
        # sha256 string
        self.pool_id=pool_id
        # discuss if the pool_id should be hashed with the object in a real case
        self.hash=hashlib.sha256(str(id).encode(encoding)).hexdigest()
        # corruption rate from 0.01 to 0.2
        self.corruption_rate=corruption_rate

        self.is_corruped = False
    def to_string(self):
        return "id: " + str(self.id) + " sha256: " + self.hash + " corruption_rate: " + str(self.corruption_rate)
        
objects = [Object(i,random.uniform(0,p)) for i in range(0,N)]

for i in range(0,3):
    print(objects[i].to_string())

assert objects[2].hash == hashlib.sha256("2".encode(encoding)).hexdigest()


id: 0 sha256: 5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9 corruption_rate: 0.03967356583585622
id: 1 sha256: 6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b corruption_rate: 0.011270764764913467
id: 2 sha256: d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35 corruption_rate: 0.029984698210919383


In [17]:
# RQ 2
# TODO finding the optimal pool size "k" with a bernoulli experiment based on the corruption rate
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8242460/
import math
def optimal_size(p):
    # https://www.sciencedirect.com/science/article/pii/S1201971220306925
    #poolsize = 1.24* p/N ^-0.466
    #return 100
    return math.ceil(1.24 * math.pow(p,-0.466))
k = optimal_size(prevalence/N)
print("Optimal poolsize {} with prevalence {} in N={}".format(k,prevalence,N))
print("Net analyses required for one object: {}".format(1/k-math.pow((1-p),k)+1))

Optimal poolsize 4 with prevalence 1.0 in N=10
Net analyses required for one object: 0.5939


In [18]:
# import matplotlib.pyplot as plt
# x = np.linspace(1,500,500)
# y= optimal_size(x/1000)
# plt.plot(x/1000,y,color="black")
# plt.plot(x[np.argmax(y<2.00)]/1000,2.00,"ro")
# plt.text(x[np.argmax(y<2.00)]/1000,4.00,"{}%".format(x[np.argmax(y<2.00)]/1000*100))
# plt.text(-0.07,1.8,"k=2",color="r")
# plt.axhline(y=2,color="r")
# plt.plot()

In [19]:
# creation of J pools, fill them with objects and assign each object to a pool
class Pool:
    def __init__(self,objects,id=-1,transaction=""):
        # list of objects in the pool
        self.objects=objects
        # sha256 root hash of the hash-list
        self.hash=hashlib.sha256("".join([obj.hash for obj in objects]).encode(encoding)).hexdigest()
        # reference to the pool, integer from 1 - inf 
        self.id=id
        # transaction hash on the ethereum blockchain
        self.transaction=transaction
    def to_string(self):
        return "PoolId: " + str(self.id) + " with " + str(len(self.objects)) +" objects in pool"

test_n = 9
testpool = Pool(objects[0:test_n])
hashlist =""
for i in range(test_n):
    hashlist+=hashlib.sha256(str(i).encode(encoding)).hexdigest()

assert testpool.hash == hashlib.sha256(hashlist.encode(encoding)).hexdigest()

In [20]:
# Pool Creation
# https://www.geeksforgeeks.org/break-list-chunks-size-n-python/
# split the list of pools into equal chunks of size k, the last pool is filled with remainders e.g. N= 14 k=3 = [3,3,3,3,2]
#pools = [Pool(i,objects[i * k:(i + 1) * k]) for i in range((len(objects) + k - 1) // k )]

pools=[]
for i in range((len(objects) + k - 1) // k ):
    # assign pool id to each object in the pool
    for obj in objects[i * k:(i + 1) * k]:
        obj.pool_id=i
    pools.append(Pool(objects[i * k:(i + 1) * k],i))

print("{} objects distributed in {} pools with size={} last pool with size={}".format(N,len(pools),k,len(pools[len(pools)-1].objects)))
assert pools[0].id == 0
assert len(pools[0].objects) == len(pools[1].objects)
assert pools[len(pools)-1].hash

10 objects distributed in 3 pools with size=4 last pool with size=2


In [21]:
ganache_url = 'http://127.0.0.1:7545'
contract_address="0x62740793ab9cd41d39B2fbA837dD19B9358Fb85b"
w3 = web3.Web3(web3.HTTPProvider(ganache_url))

sender = w3.eth.accounts[0]
balance = w3.fromWei(w3.eth.get_balance(sender),"ether")
tx_count = w3.eth.getTransactionCount(sender)
print("Transaction Count {}".format(tx_count))
print("ETH Balance: {}".format(balance))
print("Sender Account: {}".format(sender))
compiled_contract_path = '../sol/build/contracts/FixityStorage.json'
# check contract address if this cell fails
deployed_contract_address = w3.toChecksumAddress(contract_address)
print("Contract Deployed at: {}".format(deployed_contract_address))

with open(compiled_contract_path) as file:
    contract_json = json.load(file)  # load contract info as JSON
    contract_abi = contract_json['abi']  # fetch contract's abi - necessary to call its functions

# Fetch deployed contract reference
contract = w3.eth.contract(address=deployed_contract_address, abi=contract_abi)
print("Contract Functions: {}".format(contract.all_functions()))

Transaction Count 159
ETH Balance: 99.7707378677
Sender Account: 0x18648B486Bd6B771DB957590E988A2464F22BfCd
Contract Deployed at: 0x62740793ab9cd41d39B2fbA837dD19B9358Fb85b
Contract Functions: [<Function getPoolHash(uint32)>, <Function setPoolHash(uint32,bytes32)>]


In [22]:
contract.functions.setPoolHash(0,pools[0].hash).transact()

ValueError: {'message': 'from not found; is required', 'code': -32000, 'data': {'stack': 'TXRejectedError: from not found; is required\n    at StateManager.queueTransaction (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\lib\\statemanager.js:345:14)\n    at GethApiDouble.eth_sendTransaction (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\lib\\subproviders\\geth_api_double.js:325:14)\n    at GethApiDouble.handleRequest (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\lib\\subproviders\\geth_api_double.js:109:10)\n    at next (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\web3-provider-engine\\index.js:136:18)\n    at GethDefaults.handleRequest (C:\\Progra

In [ ]:
contract.functions.setPoolHash(0,pools[0].hash)

<Function setPoolHash(uint32,bytes32) bound to (0, '44e7a7f7eae93fd3e0d52cfd81347de28c7f9312c6ec662617320e269a4243d9')>

In [ ]:
gasPrice = w3.toWei('50', 'gwei')
gasPrice
eth_amount = w3.fromWei(gasPrice,"ether")
eth_price = 4000
gas_amount = 42
eth_amount*eth_price*42418


Decimal('8.48360000')

In [ ]:
txdata="0x608060405234801561001057600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555061019f806100616000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c8063178d29291461003b57806331bc18ac14610079575b600080fd5b6100776004803603604081101561005157600080fd5b81019080803563ffffffff169060200190929190803590602001909291905050506100c1565b005b6100ab6004803603602081101561008f57600080fd5b81019080803563ffffffff169060200190929190505050610142565b6040518082815260200191505060405180910390f35b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461011b57600080fd5b806000808463ffffffff1663ffffffff168152602001908152602001600020819055505050565b60008060008363ffffffff1663ffffffff16815260200190815260200160002054905091905056fea265627a7a72315820d37d9937700027df13ab8e0026a627c0742cd896e194cb80b4fc7547123f436764736f6c63430005100032"
bytesize=len(txdata.encode("UTF-8"))
print(bytesize)
bytesize*200

1026


205200

In [ ]:
deployed_bytecode="0x608060405234801561001057600080fd5b50600436106100365760003560e01c8063178d29291461003b57806331bc18ac14610079575b600080fd5b6100776004803603604081101561005157600080fd5b81019080803563ffffffff169060200190929190803590602001909291905050506100c1565b005b6100ab6004803603602081101561008f57600080fd5b81019080803563ffffffff169060200190929190505050610142565b6040518082815260200191505060405180910390f35b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461011b57600080fd5b806000808463ffffffff1663ffffffff168152602001908152602001600020819055505050565b60008060008363ffffffff1663ffffffff16815260200190815260200160002054905091905056fea265627a7a72315820d37d9937700027df13ab8e0026a627c0742cd896e194cb80b4fc7547123f436764736f6c63430005100032"
bytesize=len(deployed_bytecode.encode("UTF-8"))
print(bytesize)
bytesize*200

832


166400

In [ ]:
eth_amount

Decimal('5E-8')

In [ ]:
gas = 2000000
gasPrice = w3.toWei('50', 'gwei')

metaTx = {
    #"nonce":w3.eth.getTransactionCount(sender) nonce is set on transaction call
    "from":sender,
    "to":deployed_contract_address,
    "gas": gas,
    "gasPrice": gasPrice
}
print(metaTx)
print(w3.eth.getTransactionCount(sender))

{'from': '0x18648B486Bd6B771DB957590E988A2464F22BfCd', 'to': '0x62740793ab9cd41d39B2fbA837dD19B9358Fb85b', 'gas': 2000000, 'gasPrice': 50000000000}
158


In [29]:
tx = contract.functions.setPoolHash(600000,pools[2].hash).buildTransaction(
    {
        "nonce":w3.eth.getTransactionCount(sender),
        "gas": 50000,
        "gasPrice": 9350000000
    }
)
data = tx["data"]
data = "0x178d292900000000000000000000000000000000000000000000000000000000000000001760d27083f6e2d1c46a65938c03a0c52dccf55cb4eb68a720e6efe3a8851f78"
data.count("0")
len(data)-data.count("0")
4 * 70 + 16 * 68

1368

In [35]:
21000 + 32000 + 200 * 832 + 226 * 4 + 800 * 16 

233104

In [28]:
42368 * 0.00000005 * 4000

8.4736

In [23]:
zero_arr=[]
non_zero_arr=[]
for i in range(0,100):
    tx = contract.functions.setPoolHash(600000,pools[2].hash).buildTransaction(
        {
            "nonce":w3.eth.getTransactionCount(sender),
            "gas": 50000,
            "gasPrice": 9350000000
        }
    )
    data = tx["data"]
    zero_arr.append(data.count("0"))
    non_zero_arr.append((len(data)-data.count("0")))
    # print("zero bytes of the transaction={}*4={} gas".format(data.count("0"),data.count("0")*4))
    # print("Non-zero bytes of the transaction={}*16={} gas".format(len(data)-data.count("0"),len(data)-data.count("0")*16))

print()
print("Average zero byte cost={}".format(sum(zero_arr)/len(zero_arr)))
print("Average non zero byte cost={}".format(sum(non_zero_arr)/len(non_zero_arr)))
print("Average tx data cost={}".format(sum(zero_arr)/len(zero_arr) + sum(non_zero_arr)/len(non_zero_arr)))



Average zero byte cost=65.0
Average non zero byte cost=73.0
Average tx data cost=138.0


In [ ]:
input_data_deployment="0x608060405234801561001057600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555061019f806100616000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c8063178d29291461003b57806331bc18ac14610079575b600080fd5b6100776004803603604081101561005157600080fd5b81019080803563ffffffff169060200190929190803590602001909291905050506100c1565b005b6100ab6004803603602081101561008f57600080fd5b81019080803563ffffffff169060200190929190505050610142565b6040518082815260200191505060405180910390f35b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461011b57600080fd5b806000808463ffffffff1663ffffffff168152602001908152602001600020819055505050565b60008060008363ffffffff1663ffffffff16815260200190815260200160002054905091905056fea265627a7a72315820d37d9937700027df13ab8e0026a627c0742cd896e194cb80b4fc7547123f436764736f6c63430005100032"
zeros=input_data_deployment.count("0")
non_zeros=len(input_data_deployment)-zeros
print(zeros)
print(non_zeros)
zeros*4+non_zeros*16

226
800


13704

In [ ]:
len(data)-data.count("0")


73

In [ ]:
private_key="57b21d9e3d36c19c548091acd195c91ebe3ed27dec0fe4903d008f9af9e72f39"
signed_tx = w3.eth.account.sign_transaction(tx,private_key=private_key)
txhash=w3.eth.send_raw_transaction(signed_tx.rawTransaction)

In [ ]:
w3.eth.get_transaction_receipt(txhash)

AttributeDict({'transactionHash': HexBytes('0x6bc687701894cbc374e60b8d3bf966478a109dc352ac62c65e86397be321101d'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x3fd78804364665707e247ffa4a5eb0bb99323473cdcf2ce4804ff2e7983f936c'),
 'blockNumber': 159,
 'from': '0x18648B486Bd6B771DB957590E988A2464F22BfCd',
 'to': '0x62740793ab9cd41d39B2fbA837dD19B9358Fb85b',
 'gasUsed': 42955,
 'cumulativeGasUsed': 42955,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
metaTx.update({"nonce":w3.eth.getTransactionCount(sender)})
tx_hash = contract.functions.setPoolHash(13,pools[0].hash).transact(metaTx)

In [ ]:
contract.functions.setPoolHash(1,pools[0].hash).estimateGas()

27931

In [ ]:
23731 - 21000 - 1428

1303

In [ ]:
w3.eth.getTransaction(tx_hash)

AttributeDict({'hash': HexBytes('0xccc9120c867529a2397a3d1488c5e640f78c233c88e00ee872124a8bbd04aaf7'),
 'nonce': 150,
 'blockHash': HexBytes('0xcd030624193016414854e531ab0cfd1317bbe91ab833b2581f8cff5e05355fbd'),
 'blockNumber': 151,
 'transactionIndex': 0,
 'from': '0x18648B486Bd6B771DB957590E988A2464F22BfCd',
 'to': '0x62740793ab9cd41d39B2fbA837dD19B9358Fb85b',
 'value': 0,
 'gas': 2000000,
 'gasPrice': 50000000000,
 'input': '0x178d2929000000000000000000000000000000000000000000000000000000000000002a44e7a7f7eae93fd3e0d52cfd81347de28c7f9312c6ec662617320e269a4243d9',
 'v': 37,
 'r': HexBytes('0xa0dbd8f4741ee9215805633dd1dbfb8c10c6e0cf182b8ecb63e572e0af64dc87'),
 's': HexBytes('0x609539365ba3327a35f6e4557e938a91e515dc4309c97170936c94c14322e222')})

In [ ]:
# persist each pool on the blockchain, for each pool perform a transaction
# RQ 1 write_tx_count * gasPrice * gas or just read it from ganache
from tqdm.notebook import tqdm, trange
import time
write_tx_count = 0
read_tx_count = 0

for pool in tqdm(pools):
    metaTx.update({"nonce":w3.eth.getTransactionCount(sender)})
    tx_hash = contract.functions.setPoolHash(pool.id,pool.hash).transact(metaTx)
    write_tx_count = write_tx_count + 1 
    #print("Persisting pool "+str(pool.id)+" with hash: "+str(pool.hash)+" in transaction " + tx_hash.hex() + " succeeded")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
b = w3.fromWei(w3.eth.get_balance(sender),"ether")
b

Decimal('99.78265354')

In [ ]:
metaTx.update({"nonce":w3.eth.getTransactionCount(sender)})
tx_hash = contract.functions.setPoolHash(pool.id,pool.hash).transact(metaTx)

In [ ]:
w3.eth.getTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0x1ff96b1e922eae8f5b913aa713e27cd6eb4e4a87a52a2eba63b16826928fea20'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x8becfb6b2af1681a1d7105d8133ec88dea4a71ff95b4673de2ea8b653a39279a'),
 'blockNumber': 147,
 'from': '0x18648B486Bd6B771DB957590E988A2464F22BfCd',
 'to': '0x62740793ab9cd41d39B2fbA837dD19B9358Fb85b',
 'gasUsed': 23731,
 'cumulativeGasUsed': 23731,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
contract.events.Fixity().processReceipt(w3.eth.getTransactionReceipt(tx_hash))[0]["args"].poolId

ABIEventFunctionNotFound: ("The event 'Fixity' was not found in this contract's abi. ", 'Are you sure you provided the correct contract abi?')

In [ ]:
print(b - w3.fromWei(w3.eth.get_balance(sender),"ether"))

0.00483700


In [ ]:
# test if persistence on the blockchain was successfull
pool_id = pools[0].id
poolHashBytes = contract.functions.getPoolHash(pool_id).call()
read_tx_count = read_tx_count + 1
assert poolHashBytes.hex() == pools[pool_id].hash

AssertionError: 

In [ ]:
# "ingest" objects into the "archive"
import random

class Archive:
    def __init__(self,objects):
        self.objects=objects

    def retrieveObj(self,id):
        return next(obj for obj in objects if obj.id == id)

    def get_objects_by_pool_id(self,pool_id):
        return [obj for obj in self.objects if obj.pool_id == pool_id]
    
    def get_sample(self,n):
        return random.sample(self.objects,n)

    def corrupt(self,p):
        for obj in self.objects:
            if(random.uniform(0, 1)<p):
                obj.hash=hashlib.sha256((str(obj.id) + "x").encode(encoding)).hexdigest()
                obj.is_corruped=True

    def repair(self,pool_id):
        global write_tx_count
        write_tx_count=write_tx_count+1
        global metaTx
        metaTx.update({"nonce":w3.eth.getTransactionCount(sender)})
        contract.functions.setPoolHash(pool.id,pool.hash).transact(metaTx)
        # TODO what happens if a corrupt pool was found
        return 0
    def clean(self):
        print("Cleanup archive")

archive = Archive(objects)

assert archive.objects[k*2].pool_id == 2 
assert Pool(archive.get_objects_by_pool_id(2)).hash == pools[2].hash
assert objects[2].hash==archive.retrieveObj(2).hash
# write transactions have to be exactly the number of pools at this stage
assert write_tx_count == len(pools)

In [ ]:
print("Write transactions after ingest: " + str(write_tx_count))

Write transactions after ingest: 3


In [ ]:
sample = archive.retrieveObj(0)
assert sample.pool_id==0
pool_of_sample = Pool(archive.get_objects_by_pool_id(sample.pool_id))
assert pool_of_sample.hash == pools[0].hash
pool_in_blockchain = contract.functions.getPoolHash(sample.pool_id).call()
assert pool_of_sample.hash == pool_in_blockchain.hex()

In [ ]:
# update, zuerst schauen ob de rpool passt
id = 0
original_hash = archive.objects[id].hash
archive.objects[id].hash=original_hash

archive_pool = Pool(archive.get_objects_by_pool_id(archive.objects[id].pool_id),id=id)
pool_in_blockchain = contract.functions.getPoolHash(archive.objects[id].pool_id).call()
assert archive_pool.hash == pool_in_blockchain.hex()

# falls ja das objekt ändern
archive.objects[id].hash=original_hash+"x"
# erneut die objekte aus dem pool im archiv holen und auf der blockchain persistieren
updated_archive_pool = Pool(archive.get_objects_by_pool_id(archive.objects[id].pool_id),id=id)
assert updated_archive_pool.hash!=pool_in_blockchain.hex()
assert updated_archive_pool.id==id

metaTx.update({"nonce":w3.eth.getTransactionCount(sender)})
tx_hash = contract.functions.setPoolHash(updated_archive_pool.id,updated_archive_pool.hash).transact(metaTx)
write_tx_count = write_tx_count + 1 

# get the hash with pool id
updated_pool_in_blockchain = contract.functions.getPoolHash(archive.objects[id].pool_id).call()
assert pool_in_blockchain.hex() != updated_pool_in_blockchain.hex()

In [ ]:
archive.corrupt(p)

In [ ]:
print("Write transactions before cleaning: " + str(write_tx_count))
# repair every object in the archive
already_cleaned_pool_ids = set()
corrupted_objects_count = 0
for obj in tqdm(archive.objects):
    pool_of_sample = Pool(archive.get_objects_by_pool_id(obj.pool_id))
    pool_in_blockchain = contract.functions.getPoolHash(obj.pool_id).call()

    # is the local pool hash the same as the one in the blockchain? and make sure to not double repair a pool
    if(pool_of_sample.hash != pool_in_blockchain.hex() and obj.pool_id not in already_cleaned_pool_ids):
        write_tx_count=write_tx_count+1
        metaTx.update({"nonce":w3.eth.getTransactionCount(sender)})
        contract.functions.setPoolHash(pool.id,pool.hash).transact(metaTx)
        already_cleaned_pool_ids.add(obj.pool_id)

print("Number of Distinct Cleaned Pools: {}".format(len(already_cleaned_pool_ids)))
print("Write transactions after cleaning: {}".format(write_tx_count))


Write transactions before cleaning: 4


  0%|          | 0/10 [00:00<?, ?it/s]

Number of Distinct Cleaned Pools: 0
Write transactions after cleaning: 4


In [ ]:
fin_balance = w3.fromWei(w3.eth.get_balance(sender),"ether")
fin_tx_count = w3.eth.getTransactionCount(sender)
print("Optimal poolsize {} with prevalence {} in N={}".format(k,prevalence,N))
print("{} objects distributed in {} pools with size={} last pool with size={}".format(N,len(pools),k,len(pools[len(pools)-1].objects)))
print("Transaction Count: {} ".format(fin_tx_count - tx_count))
print("Total Cost=ETH {} for {} transactions ".format((balance - fin_balance),(fin_tx_count - tx_count)))
print("Theoretical amount of write transactions: {} with N={} + prevalence={}".format((N+prevalence),N,prevalence))
print("Repairing transactions={}, versus naive reparing transactions prevalence={}".format(fin_tx_count-tx_count-len(pools),prevalence))
print("Number of Distinct Cleaned Pools: {}".format(len(already_cleaned_pool_ids)))



Optimal poolsize 4 with prevalence 1.0 in N=10
10 objects distributed in 3 pools with size=4 last pool with size=2
Transaction Count: 4 
Total Cost=ETH 0.00564600 for 4 transactions 
Theoretical amount of write transactions: 11.0 with N=10 + prevalence=1.0
Repairing transactions=1, versus naive reparing transactions prevalence=1.0
Number of Distinct Cleaned Pools: 0


In [ ]:
# TODO optimal pool size!!!!
# TODO make a pretty result table 
# TODO experiment on online network 

In [ ]:
# TODO nächstes mal auf ropsten testnet deployen und schauen was ich für einen throughput habe ud die kosten logge
# simulieren soll ich den throughput und die kosten kann ich vorberechnen
# rauber intressiert auch wie der thorughput vorberechnet ist und danach wie groß der unterschied zum tatsächliche throughput ist